In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import gcsfs
from io import BytesIO
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [2]:
basic_model = tf.keras.models.load_model('C:/Users/koust_fiminqv/Desktop/thesis/models/basic_model.h5')

In [2]:
tuned_model = tf.keras.models.load_model('C:/Users/koust_fiminqv/Desktop/thesis/models/tuned_model1.h5')

In [3]:
X_test = np.load('X_test.npy')

In [4]:
X_test.shape

(174, 224, 224, 3)

In [5]:
y_test = np.load('y_test.npy')

In [36]:
basic_model.evaluate(X_test,y_test)

6/6 [==============================] - 2s 311ms/step - loss: 1197.7357


1197.7357177734375

In [6]:
tuned_model.evaluate(X_test,y_test)

6/6 [==============================] - 1s 182ms/step - loss: 14847.3418


14847.341796875

In [9]:
tuned_model.get_metrics_result()

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=14847.342>}

In [7]:
tuned_prediction = tuned_model.predict(X_test)

6/6 [==============================] - 7s 1s/step


In [21]:
basic_predictions = basic_model.predict(X_test)

6/6 [==============================] - 2s 344ms/step


In [22]:
tuned_prediction[0]

array([ 13.604889,  18.875927,  88.218216, 102.87601 ], dtype=float32)

In [23]:
basic_predictions[0]

array([ 21.185123,  28.662754, 212.56853 , 221.38307 ], dtype=float32)

In [24]:
y_test[0]

array([  4.17576565,   0.        , 205.50732357, 211.77097204])

In [25]:
def display_image_with_bbox(image, bbox):
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[3], bbox[2], 
                                      linewidth=2, edgecolor='r', facecolor='none'))
    plt.axis('off')
    plt.show()

In [26]:
def load_and_preprocess_testimage(path, target_size=(224, 224)):
    fs = gcsfs.GCSFileSystem(project='thesis-428619')
    path = path.replace("gs://", "")
    with fs.open(path, 'rb') as f:
        img_bytes = f.read()
        img = load_img(BytesIO(img_bytes), target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  
        return img, img_array

In [31]:
y_test[0][0]

4.175765645805592

In [32]:
actual_bounding_box = [y_test[0][0], y_test[0][1], y_test[0][2], y_test[0][3]]